In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
import json

def load_plm(model_name):
  # 确保使用的是AutoModelForMaskedLM来加载掩码语言模型
  tokenizer = AutoTokenizer.from_pretrained(f'/home/zhihao/.cache/huggingface/hub/{model_name}', trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(f'/home/zhihao/.cache/huggingface/hub/{model_name}', trust_remote_code=True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  return tokenizer, model

In [ ]:
print(os.getcwd())
os.chdir('all_datasets/flakytest/')

In [ ]:
summary_concept = "summary_concept.xlsx"
summary = "summary.xlsx"

In [ ]:
import pandas as pd

# 读取Excel文件
df = pd.read_excel(summary)
# 提取question列
summary_questions = df['question'].tolist()


# 读取Excel文件
df = pd.read_excel(summary_concept)
# 提取question列
summary_concept_questions = df['question'].tolist()

## codellama

In [ ]:
tokenizer, model = load_plm('CodeLlama-13b-Instruct-hf')

In [ ]:
def get_answer(prompt):
  inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_length=4096, top_k=1)
  response = tokenizer.decode(outputs[0])

  return response

In [ ]:
def extract_answer(text):
    start_marker = "Here's your answer: \n\n"
    end_marker = "`\n"
    start_index = text.find(start_marker)
    if start_index != -1:
        start_index += len(start_marker) + 1
        end_index = text.find(end_marker, start_index)
        if end_index != -1:
            return text[start_index:end_index].strip()
    return None

In [ ]:
def test():
  with open('test.txt', 'r') as f:
    content = f.read()
    answer = extract_answer(content)
    print(answer)

In [ ]:
test()

In [ ]:
# 输出文件路径
import csv
output_file_path = 'codellama_summary.csv'

with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['question', 'answer'])  # 写入标题行

    for question in summary_questions:
        question_with_prompt = question + "\nHere's your answer: "
        res = get_answer(question_with_prompt)  # 假设这是获取答案的函数
        answer = extract_answer(res)

        # 写入每一行数据
        writer.writerow([question, answer])

# 重复同样的过程对summary_concept_questions处理
output_file_path_concept = 'codellama_summary_concept.csv'

with open(output_file_path_concept, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['question', 'answer'])  # 写入标题行

    for question in summary_concept_questions:
        question_with_prompt = question + "\nHere's your answer: "
        res = get_answer(question_with_prompt)  # 假设这是获取答案的函数
        answer = extract_answer(res)

        # 写入每一行数据
        writer.writerow([question, answer])

print(f"新数据已保存到 {output_file_path} 和 {output_file_path_concept}")


## starchat

In [ ]:
tokenizer, model = load_plm('starchat-alpha')

In [ ]:
def test():
  with open('test.txt', 'w') as f:
    for question in summary_questions:
      question_with_prompt = question + "\nHere's your answer: "
      res = get_answer(question_with_prompt)  # 假设这是获取答案的函数
      f.write(res)
      break

In [ ]:
def extract_answer(text):
    start_marker = "Here's your answer:  `"
    end_marker = "`<|end|>"
    start_index = text.find(start_marker)
    if start_index != -1:
        start_index += len(start_marker)
        end_index = text.find(end_marker, start_index)
        if end_index != -1:
            return text[start_index:end_index].strip()
    return None

In [ ]:
def test():
  with open('test.txt', 'r') as f:
    content = f.read()
    answer = extract_answer(content)
    print(answer)

In [ ]:
test()

In [ ]:
# 输出文件路径
import csv
output_file_path = 'starchat_summary.csv'

with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['question', 'answer'])  # 写入标题行

    for i, question in enumerate(summary_questions):
        question_with_prompt = question + "\nHere's your answer: "
        res = get_answer(question_with_prompt)  # 假设这是获取答案的函数
        answer = extract_answer(res)
        with open(f'starchat/{i}.txt', 'w') as f:
          f.write(res)
        # 写入每一行数据
        writer.writerow([question, answer])

# 重复同样的过程对summary_concept_questions处理
output_file_path_concept = 'starchat_summary_concept.csv'

with open(output_file_path_concept, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['question', 'answer'])  # 写入标题行

    for i, question in enumerate(summary_concept_questions):
        question_with_prompt = question + "\nHere's your answer: "
        res = get_answer(question_with_prompt)  # 假设这是获取答案的函数
        answer = extract_answer(res)
        with open(f'starchat/{i}_concept.txt', 'w') as f:
          f.write(res)

        # 写入每一行数据
        writer.writerow([question, answer])

print(f"新数据已保存到 {output_file_path} 和 {output_file_path_concept}")


In [ ]:
def extract_answer2(text):
    start_marker = "Here's your answer: \n"
    end_marker = "<|end|>"
    start_index = text.find(start_marker)
    if start_index != -1:
        start_index += len(start_marker)
        end_index = text.find(end_marker, start_index)
        if end_index != -1:
            return text[start_index:end_index].strip()
    return None

In [ ]:
import csv

output_file_path = 'starchat_summary_concept.csv'
temp_file_path = 'temp_concept.csv'

# 读取CSV文件并找到空答案的行
rows_to_update = []
with open(output_file_path, 'r', newline='', encoding='utf-8') as csvfile, \
      open(temp_file_path, 'w', newline='', encoding='utf-8') as temp_file:
    
    reader = csv.reader(csvfile)
    header = next(reader)  # 跳过标题行

    writer = csv.writer(temp_file)
    writer.writerow(['question', 'answer'])  # 写入标题行

    for i, row in enumerate(reader):
        question, answer = row
        if answer:
            print(f'{i}: {answer} (skipped)')
            # 写入每一行数据
            writer.writerow([question, answer])

        if not answer:  # 答案为空
            # 尝试从文本文件中读取答案
            try:
                with open(f'starchat/{i}_concept.txt', 'r') as f:
                    answer = f.read().strip()
                    answer = extract_answer2(answer)
                    print(f'{i}: {answer}')
                    # 写入每一行数据
                    writer.writerow([question, answer])
                # if answer:
                    # rows_to_update.append((i, question, answer))
            except FileNotFoundError:
                print(f"File not found for question {i}: {question}")

# # 用更新过的文件替换原文件
import shutil
shutil.move(temp_file_path, output_file_path)
